# **IMPORT MODULES**

In [1]:
# Install + Imports

import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

# **LOAD THE DATA**

In [2]:
# Dataset Path

STEERING_PATH = "/kaggle/input/training-car/data"

CSV_PATH = os.path.join(STEERING_PATH, "driving_log.csv")

print("STEERING_PATH:", STEERING_PATH)
print("CSV_PATH:", CSV_PATH)

print("CSV Exists:", os.path.exists(CSV_PATH))

STEERING_PATH: /kaggle/input/training-car/data
CSV_PATH: /kaggle/input/training-car/data/driving_log.csv
CSV Exists: True


In [3]:
# Load CSV

df = pd.read_csv(CSV_PATH)

In [4]:
df.head()

,center,left,right,steering,throttle,brake,speed
0,IMG/center_2016_12_01_13_30_48_287.jpg,IMG/left_2016_12_01_13_30_48_287.jpg,IMG/right_2016_12_01_13_30_48_287.jpg,0.0,0.0,0.0,22.148290
1,IMG/center_2016_12_01_13_30_48_404.jpg,IMG/left_2016_12_01_13_30_48_404.jpg,IMG/right_2016_12_01_13_30_48_404.jpg,0.0,0.0,0.0,21.879630
2,IMG/center_2016_12_01_13_31_12_937.jpg,IMG/left_2016_12_01_13_31_12_937.jpg,IMG/right_2016_12_01_13_31_12_937.jpg,0.0,0.0,0.0,1.453011
3,IMG/center_2016_12_01_13_31_13_037.jpg,IMG/left_2016_12_01_13_31_13_037.jpg,IMG/right_2016_12_01_13_31_13_037.jpg,0.0,0.0,0.0,1.438419
4,IMG/center_2016_12_01_13_31_13_177.jpg,IMG/left_2016_12_01_13_31_13_177.jpg,IMG/right_2016_12_01_13_31_13_177.jpg,0.0,0.0,0.0,1.418236


In [5]:
df.columns

Index(['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed'], dtype='object')

In [6]:
print("Total rows:", len(df))

Total rows: 8036


# **MODEL BUILDING**

In [7]:
# Dataset Class

class SteeringDataset(Dataset):
    def __init__(self, df, base_path):
        self.df = df
        self.base_path = base_path

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        img_name = row["center"]       # example: IMG/center_xxx.jpg
        steering = float(row["steering"])

        img_path = os.path.join(self.base_path, img_name)

        img = cv2.imread(img_path)
        if img is None:
            raise FileNotFoundError(f"Image not found: {img_path}")

        # crop bottom part (road region)
        img = img[-150:]

        # resize to NVIDIA model input
        img = cv2.resize(img, (200, 66))

        # normalize
        img = img.astype(np.float32) / 255.0

        # BGR -> RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # HWC -> CHW
        img = np.transpose(img, (2, 0, 1))

        img_tensor = torch.tensor(img, dtype=torch.float32)
        steering_tensor = torch.tensor([steering], dtype=torch.float32)

        return img_tensor, steering_tensor


In [8]:
# NVIDIA CNN Model

class NvidiaSteeringModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 24, kernel_size=5, stride=2)
        self.conv2 = nn.Conv2d(24, 36, kernel_size=5, stride=2)
        self.conv3 = nn.Conv2d(36, 48, kernel_size=5, stride=2)
        self.conv4 = nn.Conv2d(48, 64, kernel_size=3, stride=1)
        self.conv5 = nn.Conv2d(64, 64, kernel_size=3, stride=1)

        self.fc1 = nn.Linear(64 * 1 * 18, 1164)
        self.fc2 = nn.Linear(1164, 100)
        self.fc3 = nn.Linear(100, 50)
        self.fc4 = nn.Linear(50, 10)
        self.fc5 = nn.Linear(10, 1)

        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))

        x = x.view(x.size(0), -1)

        x = F.relu(self.fc1(x))
        x = self.dropout(x)

        x = F.relu(self.fc2(x))
        x = self.dropout(x)

        x = F.relu(self.fc3(x))
        x = self.dropout(x)

        x = F.relu(self.fc4(x))
        x = self.dropout(x)

        return self.fc5(x)


In [9]:
# Create Dataset + Train/Val Split

dataset = SteeringDataset(df, STEERING_PATH)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])

print("Train samples:", len(train_ds))
print("Val samples:", len(val_ds))


Train samples: 6428
Val samples: 1608


In [10]:
# DataLoaders

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False)

In [11]:
# Training Setup

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model = NvidiaSteeringModel().to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

SAVE_DIR = "/kaggle/working/steering_model"
os.makedirs(SAVE_DIR, exist_ok=True)

best_val_loss = float("inf")
EPOCHS = 10  # ----> Do it for 30 epochs for time purpose I give 10 epochs

Using device: cuda


In [12]:
# Training Loop + Save Best Model

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0

    for imgs, angles in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} Training"):
        imgs, angles = imgs.to(device), angles.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)

        loss = criterion(outputs, angles)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    model.eval()
    val_loss = 0

    with torch.no_grad():
        for imgs, angles in tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} Validation"):
            imgs, angles = imgs.to(device), angles.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, angles)
            val_loss += loss.item()

    val_loss /= len(val_loader)

    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    print(f"Train Loss: {train_loss:.6f}")
    print(f"Val Loss  : {val_loss:.6f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), os.path.join(SAVE_DIR, "steering_model.pt"))
        print("✅ Saved Best Model!")


Epoch 1/10 Validation: 100%|██████████| 26/26 [00:13<00:00,  1.95it/s]



Epoch 1/10
Train Loss: 0.030934
Val Loss  : 0.019348
✅ Saved Best Model!


Epoch 2/10 Validation: 100%|██████████| 26/26 [00:03<00:00,  8.35it/s]



Epoch 2/10
Train Loss: 0.021982
Val Loss  : 0.018743
✅ Saved Best Model!


Epoch 3/10 Validation: 100%|██████████| 26/26 [00:02<00:00,  9.05it/s]


...


Epoch 8/10
Train Loss: 0.013565
Val Loss  : 0.011838


Epoch 9/10 Validation: 100%|██████████| 26/26 [00:02<00:00,  9.16it/s]



Epoch 9/10
Train Loss: 0.012978
Val Loss  : 0.010926
✅ Saved Best Model!


Epoch 10/10 Validation: 100%|██████████| 26/26 [00:02<00:00,  8.84it/s]


Epoch 10/10
Train Loss: 0.012855
Val Loss  : 0.010784
✅ Saved Best Model!


In [13]:
# DOWNLOAD THE MODEL FROM KAGGLE WORKING